In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

In [2]:
# Load the data
train_base_sum = pd.read_csv("./data/train/base_train_sum.csv", encoding="gbk")
train_knowledge_sum = pd.read_csv("./data/train/knowledge_train_sum.csv", encoding="gbk")
train_money_report_sum = pd.read_csv("./data/train/money_report_train_sum.csv", encoding="gbk")
train_year_report_sum = pd.read_csv("./data/train/year_report_train_sum.csv", encoding="gbk")
verify_base = pd.read_csv("./data/verify/base_verify1.csv", encoding="gbk")
verify_money_info = pd.read_csv("./data/verify/money_information_verify1.csv", encoding="gbk")
verify_paient_info = pd.read_csv("./data/verify/paient_information_verify1.csv", encoding="gbk")
verify_year_report = pd.read_csv("./data/verify/year_report_verify1.csv", encoding="gbk")

In [3]:
train_base_sum

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag
0,28,2007.0,2050.0,交通运输业,福建,农民专业合作社,企业法人,NaN,1.0
1,230,2008.0,3360.0,服务业,广东,农民专业合作社,企业法人,1.00,1.0
2,429,2005.0,9670.0,工业,江西,集体所有制企业,自然人,0.75,1.0
3,693,2011.0,8360.0,社区服务,山东,股份有限公司,企业法人,0.98,1.0
4,727,2001.0,8720.0,零售业,江西,股份有限公司,企业法人,0.54,1.0
...,...,...,...,...,...,...,...,...,...
15045,5978029,2014.0,460.0,交通运输业,广西,农民专业合作社,企业法人,0.71,NaN
15046,5978030,2004.0,1140.0,商业服务业,湖南,有限责任公司,自然人,0.95,NaN
15047,5978031,2008.0,3290.0,交通运输业,湖南,集体所有制企业,自然人,0.63,NaN
15048,5978032,2009.0,6060.0,工业,福建,农民专业合作社,企业法人,0.51,NaN


In [10]:
def merge_base_knowledge():
    df = pd.merge(train_base_sum, train_knowledge_sum, on='ID')
    return df
test_df = merge_base_knowledge()
# flag = 1 暂定表示为僵尸企业
# flag除了1之外都是缺失值，使用0补齐，表示为非僵尸企业
test_df.loc[:,'flag'] = test_df['flag'].fillna(0)
test_df

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag,专利,商标,著作权
0,28,2007.0,2050.0,交通运输业,福建,农民专业合作社,企业法人,NaN,1.0,0.0,1.0,1.0
1,230,2008.0,3360.0,服务业,广东,农民专业合作社,企业法人,1.00,1.0,0.0,0.0,0.0
2,429,2005.0,9670.0,工业,江西,集体所有制企业,自然人,0.75,1.0,1.0,0.0,0.0
3,693,2011.0,8360.0,社区服务,山东,股份有限公司,企业法人,0.98,1.0,0.0,0.0,0.0
4,727,2001.0,8720.0,零售业,江西,股份有限公司,企业法人,0.54,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15045,5978029,2014.0,460.0,交通运输业,广西,农民专业合作社,企业法人,0.71,0.0,1.0,1.0,1.0
15046,5978030,2004.0,1140.0,商业服务业,湖南,有限责任公司,自然人,0.95,0.0,0.0,0.0,0.0
15047,5978031,2008.0,3290.0,交通运输业,湖南,集体所有制企业,自然人,0.63,0.0,0.0,1.0,1.0
15048,5978032,2009.0,6060.0,工业,福建,农民专业合作社,企业法人,0.51,0.0,1.0,0.0,1.0


In [7]:
def merge_money_year(): 
    train_money_report_sum.loc[:,'year'] = train_money_report_sum['year'].fillna(method='bfill')
    train_year_report_sum.loc[:,'year'] = train_year_report_sum['year'].fillna(method='bfill')
    df = pd.merge(train_money_report_sum, train_year_report_sum, on=['ID', 'year'])
    return df
test_df = merge_money_year()
test_df

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
0,28,2015.0,0.0,0.0,0.00,0.0000,21648.0,1298.880,0.0,0.000,794.0,16400.0,28700.0,72160.0,28864.0,7216.0,-7216.0,0.0,-12300.0
1,230,2015.0,0.0,0.0,0.00,0.0000,0.0,0.000,470.4,28.224,485.0,23520.0,10080.0,115248.0,57624.0,57624.0,-11524.8,0.0,13440.0
2,429,2015.0,19340.0,1547.2,0.00,0.0000,0.0,0.000,0.0,0.000,136.0,193400.0,183730.0,502840.0,351988.0,NaN,-50284.0,0.0,9670.0
3,693,2015.0,0.0,0.0,0.00,0.0000,0.0,0.000,5350.4,321.024,534.0,133760.0,125400.0,655424.0,262169.6,196627.2,-65542.4,0.0,8360.0
4,727,2015.0,0.0,0.0,32229.12,1289.1648,0.0,0.000,0.0,0.000,375.0,366240.0,536280.0,402864.0,282004.8,161145.6,-40286.4,0.0,-170040.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45145,5978029,2017.0,0.0,0.0,0.00,0.0000,1117.8,67.068,0.0,0.000,265.0,1380.0,1840.0,3726.0,1863.0,745.2,372.6,2235.6,-460.0
45146,5978030,2017.0,0.0,0.0,0.00,0.0000,0.0,0.000,2371.2,142.272,1000.0,59280.0,58140.0,260832.0,208665.6,78249.6,-26083.2,0.0,1140.0
45147,5978031,2017.0,0.0,0.0,0.00,0.0000,0.0,0.000,2664.9,159.894,357.0,88830.0,171080.0,106596.0,42638.4,10659.6,-10659.6,0.0,-82250.0
45148,5978032,2017.0,0.0,0.0,0.00,0.0000,0.0,0.000,2908.8,174.528,572.0,145440.0,209070.0,421776.0,253065.6,168710.4,-42177.6,0.0,-63630.0


In [9]:
test_df[test_df['ID'] == 5978032]

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
29148,5978032,2015.0,0.0,0.00,25524.72,1020.9888,0.0,0.0,0.0,0.000,21.0,109080.0,103020.0,425412.0,170164.8,127623.6,170164.8,255247.2,6060.0
37148,5978032,2016.0,12726.0,1018.08,0.00,0.0000,0.0,0.0,0.0,0.000,769.0,127260.0,242400.0,407232.0,162892.8,122169.6,-40723.2,0.0,-115140.0
45148,5978032,2017.0,0.0,0.00,0.00,0.0000,0.0,0.0,2908.8,174.528,572.0,145440.0,209070.0,421776.0,253065.6,168710.4,-42177.6,0.0,-63630.0
